In [ ]:
import pandas as pd
import numpy as np
import wrds
from fredapi import Fred

In [ ]:
conn = wrds.Connection(wrds_username="nberzins")

In [50]:
conn.describe_table(library= "crsp", table='dsenames')

Approximately 115451 rows in crsp.dsenames.


,name,nullable,type,comment
0,permno,True,INTEGER,PERMNO
1,namedt,True,DATE,Names Date
2,nameendt,True,DATE,Names Ending Date
3,shrcd,True,SMALLINT,Share Code
4,exchcd,True,SMALLINT,Exchange Code
5,siccd,True,INTEGER,Standard Industrial Classification Code
6,ncusip,True,VARCHAR(8),CUSIP
7,ticker,True,VARCHAR(8),Ticker Symbol
8,comnam,True,VARCHAR(35),Company Name
9,shrcls,True,VARCHAR(4),Share Class


In [57]:
sp500 = conn.raw_sql("""
                        SELECT a.*, b.date, b.ret
                        FROM crsp.dsp500list as a, crsp.dsf as b
                        WHERE a.permno=b.permno
                            AND b.date >= a.start and b.date <= a.ending
                            AND b.date >= '01/01/2000'
                        ORDER BY date; 
                        """, date_cols=['start', 'ending', 'date'])

In [58]:
dse = conn.raw_sql("""
                        SELECT comnam, namedt, nameendt, permno
                        FROM crsp.dsenames
                        """, date_cols=['namedt', 'nameendt'])

In [74]:
fama_five = conn.raw_sql("""
                            SELECT *
                            FROM ff.fivefactors_daily as ff
                            WHERE ff.date >= '01/01/2000'
                            """)

fama_five["date"] = pd.to_datetime(fama_five["date"], format='%Y-%m-%d')

fama_five.tail(5)

,date,mktrf,smb,hml,rmw,cma,rf,umd
6284,2024-12-24,0.0111,-0.0012,-0.0005,-0.0013,-0.0037,0.00017,0.0067
6285,2024-12-26,0.0002,0.0109,-0.0019,-0.0044,0.0035,0.00017,0.0001
6286,2024-12-27,-0.0117,-0.0044,0.0056,0.0041,0.0003,0.00017,-0.0088
6287,2024-12-30,-0.0109,0.0024,0.0074,0.0055,0.0014,0.00017,0.0006
6288,2024-12-31,-0.0046,0.0031,0.0071,0.0033,0.0,0.00017,-0.0107


In [75]:
fred_api_key = "62ce35723dc6bc5a66d7276925744d91"

fred = Fred(api_key = fred_api_key)
fed_funds = fred.get_series('FEDFUNDS', observation_start='2000-01-01')

In [76]:
ffer_df = fed_funds.to_frame(name="fed_funds_rate").reset_index()

ffer_df.describe()
ffer_df = ffer_df.rename(columns={"index": "date", "fed_funds_rate": "fed_funds_rate"})

ffer_df["date"] = pd.to_datetime(ffer_df["date"], format="%Y-%d-%m")
ffer_df["date"] = ffer_df["date"].dt.strftime('%Y-%m-%d')
ffer_df["date"] = pd.to_datetime(ffer_df["date"], format="%Y-%m-%d")

ffer_df

,date,fed_funds_rate
0,2000-01-01,5.45
1,2000-02-01,5.73
2,2000-03-01,5.85
3,2000-04-01,6.02
4,2000-05-01,6.27
...,...,...
299,2024-12-01,4.48
300,2025-01-01,4.33
301,2025-02-01,4.33
302,2025-03-01,4.33


In [77]:
#Merging returns w/ company info

sp500_full = pd.merge(sp500, dse, how='left', on='permno')
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                             & (sp500_full.date<=sp500_full.nameendt)]

# Merging fama french factors and complete sp500 data

sp500_full = pd.merge(sp500_full, fama_five, how='left', on='date')

#Merging Federal Funds Rate Data 

#because only working with monthly values, have to break up the data into year and month then join on those two columns
sp500_full["month"] = sp500_full["date"].dt.month
sp500_full["year"] = sp500_full["date"].dt.year

ffer_df["month"] = ffer_df["date"].dt.month
ffer_df["year"] = ffer_df["date"].dt.year

#getting rid of unneccesary date column to avoid repeat column naames post-merge
ffer_df = ffer_df.drop(columns=["date"])

sp500_full = pd.merge(sp500_full, ffer_df, on=["year","month"], how="left")

# Removing now unnecessary month/year columns & repeat
sp500_full = sp500_full.drop(columns=["month", "year"])


In [78]:
# Date str --> Datetime conversion
sp500_full["date"] = pd.to_datetime(sp500_full["date"], format='%Y-%m-%d')

# ret, mktrf, smb, etc. --> log returns conversion
# Log-returns prove additive and will not impact future return-based factor calculations

cols_to_log = ["ret", "mktrf", "smb", "hml", "rmw", "cma", "rf"]
sp500_full[cols_to_log] = np.log1p(sp500_full[cols_to_log])

In [79]:
# Calculating Volatility

sp500_full["volatility_21d"] = (
    sp500_full.groupby("comnam")["ret"]
    .rolling(window=21, min_periods=1)
    .std()
    .reset_index(level=0, drop = True)
    .fillna(0))

# Calculating 5 day Short-term reversal

sp500_full["5d_reversal"] = -(
    sp500_full.groupby("comnam")["ret"]
    .rolling(window=5, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

# Calculating 1 day lagged short-term reversal

sp500_full["1d_lag_reversal"] = -(
    sp500_full.groupby("comnam")["ret"].shift(1)
    .fillna(0)
)

In [80]:
sp500_full.to_csv("sp500_full.csv", index = False)

sp500_full = pd.read_csv("sp500_full.csv")

In [81]:
#sp500_full = sp500_full[sp500_full["start"]]
sp500_full[sp500_full["comnam"].isin(['COASTAL CORP'])].head(40)

,permno,start,ending,date,ret,comnam,namedt,nameendt,mktrf,smb,hml,rmw,cma,rf,umd,fed_funds_rate,volatility_21d,5d_reversal,1d_lag_reversal
333,38893,1985-05-23,2001-01-29,2000-01-03,-0.032261,COASTAL CORP,1998-09-01,2001-01-29,-0.007125,-0.000600,-0.014200,-0.015114,-0.006421,0.00021,-0.0008,5.45,0.000000,0.032261,-0.000000
975,38893,1985-05-23,2001-01-29,2000-01-04,-0.039002,COASTAL CORP,1998-09-01,2001-01-29,-0.041447,0.003295,0.020391,0.004689,0.014396,0.00021,-0.0191,5.45,0.004767,0.071263,0.032261
1020,38893,1985-05-23,2001-01-29,2000-01-05,0.007547,COASTAL CORP,1998-09-01,2001-01-29,-0.000900,0.003295,0.001599,0.004092,0.011039,0.00021,-0.0049,5.45,0.025156,0.063715,0.039002
1754,38893,1985-05-23,2001-01-29,2000-01-06,0.020466,COASTAL CORP,1998-09-01,2001-01-29,-0.007327,-0.000400,0.012521,0.006479,0.012027,0.00021,-0.0149,5.45,0.029269,0.043249,-0.007547
2074,38893,1985-05-23,2001-01-29,2000-01-07,0.020056,COASTAL CORP,1998-09-01,2001-01-29,0.031596,-0.009444,-0.014302,-0.008839,-0.009646,0.00021,0.0056,5.45,0.028863,0.023193,-0.020466
2667,38893,1985-05-23,2001-01-29,2000-01-10,-0.009066,COASTAL CORP,1998-09-01,2001-01-29,0.017447,0.004988,-0.015317,-0.019897,-0.002403,0.00021,0.0200,5.45,0.025879,-0.000001,-0.020056
3259,38893,1985-05-23,2001-01-29,2000-01-11,-0.005479,COASTAL CORP,1998-09-01,2001-01-29,-0.017248,0.003295,0.008067,0.008167,0.010643,0.00021,-0.0114,5.45,0.023625,-0.033524,0.009066
3896,38893,1985-05-23,2001-01-29,2000-01-12,-0.016621,COASTAL CORP,1998-09-01,2001-01-29,-0.006924,-0.002202,0.007770,0.004191,0.009455,0.00021,-0.0065,5.45,0.022230,-0.009355,0.005479
4333,38893,1985-05-23,2001-01-29,2000-01-13,0.022100,COASTAL CORP,1998-09-01,2001-01-29,0.015775,0.004988,-0.009142,-0.017859,-0.010454,0.00021,0.0022,5.45,0.022916,-0.010989,0.016621
4968,38893,1985-05-23,2001-01-29,2000-01-14,0.048009,COASTAL CORP,1998-09-01,2001-01-29,0.011434,0.002597,-0.006521,-0.003105,-0.006320,0.00021,0.0119,5.45,0.027074,-0.038943,-0.022100


In [84]:
factor_cols = ["ret", "mktrf", "smb", "hml", "rmw", "cma", "rf", "umd", 
               "fed_funds_rate", "volatility_21d", "5d_reversal", "1d_lag_reversal"]

sp500_dly = {
    name: group[["date"] +factor_cols].reset_index(drop=True)
    for name, group in sp500_full.groupby("comnam")
}

sp500_dly["GENERAL ELECTRIC CO"]

,date,ret,mktrf,smb,hml,rmw,cma,rf,umd,fed_funds_rate,volatility_21d,5d_reversal,1d_lag_reversal
0,2000-01-03,-0.031176,-0.007125,-0.000600,-0.014200,-0.015114,-0.006421,0.00021,-0.0008,5.45,0.000000,0.031176,-0.000000
1,2000-01-04,-0.039521,-0.041447,0.003295,0.020391,0.004689,0.014396,0.00021,-0.0191,5.45,0.005901,0.070697,0.031176
2,2000-01-05,-0.005216,-0.000900,0.003295,0.001599,0.004092,0.011039,0.00021,-0.0049,5.45,0.017891,0.075912,0.039521
3,2000-01-06,0.015457,-0.007327,-0.000400,0.012521,0.006479,0.012027,0.00021,-0.0149,5.45,0.025075,0.060455,0.005216
4,2000-01-07,0.037990,0.031596,-0.009444,-0.014302,-0.008839,-0.009646,0.00021,0.0056,5.45,0.032180,0.022465,-0.015457
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6284,2024-12-24,0.014508,0.011039,-0.001201,-0.000500,-0.001301,-0.003707,0.00017,0.0067,4.48,0.020911,-0.033743,-0.003854
6285,2024-12-26,0.003842,0.000200,0.010841,-0.001902,-0.004410,0.003494,0.00017,0.0001,4.48,0.020948,-0.073506,-0.014508
6286,2024-12-27,-0.010452,-0.011769,-0.004410,0.005584,0.004092,0.000300,0.00017,-0.0088,4.48,0.020124,-0.033183,-0.003842
6287,2024-12-30,-0.011710,-0.010960,0.002397,0.007373,0.005485,0.001399,0.00017,0.0006,4.48,0.019786,-0.000041,0.010452


In [85]:
sp500_wkly = {}

for name, df in sp500_dly.items():

    #Converting to Weekly
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)

    weekly_df = df[factor_cols].resample("W-MON", label="left", closed="left").sum().reset_index()
    # weekly_df = df.resample("W-MON", label="left", closed="left")["ret"].sum().reset_index()
    
    sp500_wkly[name] = weekly_df

sp500_wkly["GENERAL ELECTRIC CO"]

,date,ret,mktrf,smb,hml,rmw,cma,rf,umd,fed_funds_rate,volatility_21d,5d_reversal,1d_lag_reversal
0,2000-01-03,-0.022465,-0.025204,-0.003856,0.006009,-0.008693,0.021395,0.00105,-0.0341,27.25,0.081046,0.260706,0.060455
1,2000-01-10,-0.002068,0.020484,0.013664,-0.015142,-0.028502,0.000921,0.00105,0.0162,27.25,0.125937,-0.168474,-0.053970
2,2000-01-17,-0.046600,0.000377,0.058498,-0.026008,-0.049815,0.001161,0.00084,0.0524,21.80,0.084597,0.146427,0.052150
3,2000-01-24,-0.072843,-0.058778,0.008962,0.016715,0.013913,0.016285,0.00105,-0.0302,27.25,0.108922,0.307622,0.029115
4,2000-01-31,0.054902,0.043595,-0.001359,-0.026716,-0.020247,-0.013999,0.00109,0.0372,28.37,0.114382,0.067784,0.017794
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,2024-12-02,-0.036789,0.010163,-0.013489,-0.030582,-0.008357,-0.013135,0.00085,0.0174,22.40,0.085487,0.092021,0.043512
1301,2024-12-09,-0.056047,-0.009292,-0.009654,-0.001631,0.024498,-0.012535,0.00085,-0.0100,22.40,0.090947,0.289278,0.047856
1302,2024-12-16,0.014116,-0.022998,-0.019514,-0.014670,-0.009697,0.006406,0.00085,0.0070,22.40,0.094542,0.104244,0.000183
1303,2024-12-23,0.011751,0.005551,-0.003306,0.001281,0.001177,0.006665,0.00068,-0.0028,17.92,0.082858,-0.159728,-0.043635


In [87]:
sp500_monthly = {}

for name, df in sp500_dly.items():

    #Converting to Monthly
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"]) # double checking date time conversion
    df.set_index("date", inplace=True)

    monthly_df = df[factor_cols].resample("MS", label="left", closed="left").sum().reset_index()
    # monthly_df = df.resample("MS", label="left", closed="left")["ret"].sum().reset_index()

    #Adding 1-yr momentum

    sp500_monthly[name] = monthly_df

sp500_monthly["GENERAL ELECTRIC CO"]

,date,ret,mktrf,smb,hml,rmw,cma,rf,umd,fed_funds_rate,volatility_21d,5d_reversal,1d_lag_reversal
0,2000-01-01,-0.146778,-0.048232,0.043297,-0.022837,-0.065625,0.044153,0.004200,0.0012,109.00,0.423448,0.579404,0.143975
1,2000-02-01,-0.009400,0.025013,0.165099,-0.085436,-0.170095,-0.016842,0.004400,0.1737,114.60,0.479096,0.300866,0.038513
2,2000-03-01,0.164789,0.047516,-0.154162,0.077688,0.113376,-0.010934,0.004600,-0.0680,134.55,0.637691,-1.009727,-0.210981
3,2000-04-01,0.010387,-0.065857,-0.052114,0.071632,0.069396,0.050555,0.004559,-0.0834,114.38,0.540578,-0.113943,-0.017174
4,2000-05-01,0.005153,-0.044568,-0.041623,0.044268,0.037907,0.009677,0.005059,-0.0700,137.94,0.482930,0.281519,0.044312
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2024-08-01,0.025636,0.015823,-0.035579,-0.011704,0.006412,0.009254,0.004839,0.0482,117.26,0.517322,-0.119041,-0.035773
296,2024-09-01,0.078426,0.017199,-0.009971,-0.026891,0.000638,-0.001640,0.004000,0.0002,102.60,0.388803,-0.422794,-0.067285
297,2024-10-01,-0.093308,-0.009547,-0.008098,0.008430,-0.015303,0.009860,0.003910,0.0365,111.09,0.410961,0.342694,0.059681
298,2024-11-01,0.058669,0.062820,0.044137,0.000962,-0.025044,-0.020240,0.004000,0.0059,92.80,0.440804,-0.189730,-0.033555


In [89]:
panel_df = []

for name, df in sp500_monthly.items():
    df = df.copy()
    df["comnam"] = name
    panel_df.append(df)
    
panel_df = pd.concat(panel_df, ignore_index=True)

,date,ret,mktrf,smb,hml,rmw,cma,rf,umd,fed_funds_rate,volatility_21d,5d_reversal,1d_lag_reversal,comnam
0,2000-01-01,0.076764,-0.048232,0.043297,-0.022837,-0.065625,0.044153,0.004200,0.0012,109.00,0.810623,-0.101544,0.009352,3COM CORP
1,2000-02-01,0.658057,0.025013,0.165099,-0.085436,-0.170095,-0.016842,0.004400,0.1737,114.60,0.939964,-2.764241,-0.529444,3COM CORP
2,2000-03-01,-0.566334,0.047516,-0.154162,0.077688,0.113376,-0.010934,0.004600,-0.0680,134.55,2.163517,1.455956,0.295883,3COM CORP
3,2000-04-01,-0.343915,-0.065857,-0.052114,0.071632,0.069396,0.050555,0.004559,-0.0834,114.38,1.305830,2.346099,0.414003,3COM CORP
4,2000-05-01,0.058478,-0.044568,-0.041623,0.044268,0.037907,0.009677,0.005059,-0.0700,137.94,1.247143,-0.112958,-0.087682,3COM CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151976,2024-08-01,0.018980,0.015823,-0.035579,-0.011704,0.006412,0.009254,0.004839,0.0482,117.26,0.391313,-0.047613,-0.005812,ZOETIS INC
151977,2024-09-01,0.062788,0.017199,-0.009971,-0.026891,0.000638,-0.001640,0.004000,0.0002,102.60,0.201923,-0.304435,-0.063142,ZOETIS INC
151978,2024-10-01,-0.086376,-0.009547,-0.008098,0.008430,-0.015303,0.009860,0.003910,0.0365,111.09,0.243775,0.342794,0.063960,ZOETIS INC
151979,2024-11-01,-0.019944,0.062820,0.044137,0.000962,-0.025044,-0.020240,0.004000,0.0059,92.80,0.341149,0.114391,0.030973,ZOETIS INC


In [93]:
X_cols = ["mktrf", "smb", "hml", "rmw", "cma", "rf"]

X = panel_df[X_cols].values
y = panel_df["ret"].values

In [94]:
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# OLS
ols = LinearRegression().fit(X,y)

# PCR
pcr = make_pipeline(StandardScaler(), PCA(n_components=min(5, X.shape[1])), LinearRegression())
pcr.fit(X,y)

# LASSO w/ cross-val
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5)).fit(X,y)

In [100]:
pd.Series(ols.coef_, index=X_cols)
ols.score(X,y)

0.23962460662012064

In [102]:
lasso_model = lasso.named_steps['lassocv']

print(pd.Series(lasso_model.coef_, index = X_cols))
lasso.score(X,y)

mktrf    0.050795
smb      0.001445
hml      0.009139
rmw      0.002946
cma      0.001262
rf       0.001082
dtype: float64


0.2396229792258533

In [ ]:
#Need to Build Different Retun-Based Factors from Returns 

#e.g. Momentum, Volatility from past 30 days, market portfolio, banking/construction (responsive to IR), IR itself,
# others can be extracxted from French Fama ---- Shoot for 10 - 15 predictors for first iteration

# *Maybe find sentiment indices*

In [ ]:
#For next week -----------------------------

# Convert to weekly + Monthly data, too much noise in daily 
# Many large positions flip before market close to avoid sitting overnight 

#Focus on predictors and run standard models in ord

In [ ]:
# Next iteration would be to just include a ton of different predictors, 